In [1]:
from google.colab import files

files.upload_file('intents.json')

Saved intents.json to /content/intents.json


In [2]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=a1e0708d08d9ac5dce9ad04b7ad9ba0a58e7e8a56c26e03c2bce4494f6880da5
  Stored in directory: /root/.cache/pip/wheels/4a/d5/f8/9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
import numpy as np
import tflearn
import tensorflow
import random
import json
import pickle

In [5]:
from nltk.stem.lancaster import LancasterStemmer
stem = LancasterStemmer() # stem of words instance

In [6]:
with open("/content/intents.json") as file:
	data = json.load(file)

In [7]:
print(data)

{'intents': [{'tag': 'bienvenida', 'patterns': ['hola', 'que tal', 'como estas'], 'responses': ['hola tu', 'ando bien']}, {'tag': 'tutorial', 'patterns': ['necesito ayuda', 'como empiezo?'], 'responses': ['hola, este es un tutorial', 'epale ya te ayudo']}, {'tag': 'adios', 'patterns': ['ya es tarde, me voy', 'bai, te escribo luego'], 'responses': ['bai bai']}, {'tag': 'lectura', 'patterns': ['estoy leyendo sobre bukouswy', 'ando leyendo'], 'responses': ['que lees?', 'suena interasante']}]}


In [8]:
words = []
labels = []
X = []
Y = []

In [9]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
      words_aux = nltk.word_tokenize(pattern)
      words.extend(words_aux)
      X.append(words_aux)
      Y.append(intent["tag"])

    if intent["tag"] not in labels:
      labels.append(intent["tag"])


In [10]:
words = [stem.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))

print(words)

[',', 'ando', 'ayud', 'bai', 'bukouswy', 'como', 'empiezo', 'es', 'escribo', 'esta', 'estoy', 'hol', 'leyendo', 'luego', 'me', 'necesito', 'que', 'sobr', 'tal', 'tard', 'te', 'voy', 'ya']


In [11]:
labels = sorted(labels)
print(labels)

['adios', 'bienvenida', 'lectura', 'tutorial']


In [12]:
training = []
output = []

In [13]:
out_empty = [0 for _ in range(len(labels))]
print(out_empty)

[0, 0, 0, 0]


In [14]:
for x, pattern in enumerate(X):
    bag = []

    wrds = [stem.stem(w) for w in pattern]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
    
    output_row = out_empty[:]
    output_row[labels.index(Y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [17]:
training = np.array(training)
output = np.array(output)

print(training.shape, output.shape)

(9, 23) (9, 4)


In [18]:
tensorflow.compat.v1.reset_default_graph()

In [19]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=32, show_metric=True)
	model.save("model.tflearn")

---------------------------------
Run id: 4CX4Y2
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 9
Validation samples: 0
--
Training Step: 1  | time: 3.251s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 9/9
--
Training Step: 2  | time: 0.007s
| Adam | epoch: 002 | loss: 0.00000 - acc: 0.2000 -- iter: 9/9
--
Training Step: 3  | time: 0.003s
| Adam | epoch: 003 | loss: 0.00000 - acc: 0.3091 -- iter: 9/9
--
Training Step: 4  | time: 0.005s
| Adam | epoch: 004 | loss: 0.00000 - acc: 0.3273 -- iter: 9/9
--
Training Step: 5  | time: 0.005s
| Adam | epoch: 005 | loss: 0.00000 - acc: 0.3315 -- iter: 9/9
--
Training Step: 6  | time: 0.005s
| Adam | epoch: 006 | loss: 0.00000 - acc: 0.3327 -- iter: 9/9
--
Training Step: 7  | time: 0.006s
| Adam | epoch: 007 | loss: 0.00000 - acc: 0.3331 -- iter: 9/9
--
Training Step: 8  | time: 0.005s
| Adam | epoch: 008 | loss: 0.00000 - acc: 0.3332 -- iter: 9/9
--
Training Step: 9  | time: 0.005s
| Adam | epoc

In [21]:
def create_bag_of_words(inp, words):
  bag = [0 for _ in range(len(words))]

  word_t = nltk.word_tokenize(inp)
  word_t = [stem.stem(word.lower()) for word in word_t]

  for se in word_t:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)



In [25]:
def chatBot(message):
  results = model.predict([create_bag_of_words(message,words)])[0]
  results_index = np.argmax(results)
  tag = labels[results_index]
  if results[results_index] > 0.5:
    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
    print(random.choice(responses))
    print("\n")
  else:
    print("No entiendo lo que dices sorry")


In [27]:
chatBot("hola amigo mio")
print(create_bag_of_words("hola", words))

ando bien


[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
